**классификатор текстов LSTM на Keras+TensorFlow**

Евгений Борисов <borisov.e@solarl.ru>

In [1]:
# https://habr.com/ru/company/dca/blog/274027/
# http://neuro.compute.dtu.dk/wiki/Sentiment_analysis#Corpora
# http://help.sentiment140.com/for-students/
# http://study.mokoron.com

## Библиотеки

In [2]:
import numpy as np
import pandas as pd
pd.options.display.max_colwidth = 200  
import re
import gzip

In [3]:
def pp(d): return "{:,.0f}".format(d).replace(",", " ")
def ppr(d): print('записей:', pp(len(d)) )  

## Данные

In [4]:
ff = ['id', 'tdate', 'tmane', 'ttext', 'ttype', 'trep', 'tfav', 'tstcount', 'tfol', 'tfrien', 'listcount','unk']

In [5]:
neg = pd.read_csv('../data/text/twit/negative.csv.gz',sep=';',header=None)
ppr(neg)
neg.columns = ff

записей: 111 923


In [6]:
pos = pd.read_csv('../data/text/twit/positive.csv.gz',sep=';')
ppr(pos)
pos.columns = ff

записей: 114 910


In [7]:
data = pd.concat([pos,neg],sort=False)[['id','ttext', 'ttype']]
ppr(data)

записей: 226 833


In [8]:
data.sample(10)

,id,ttext,ttype
3825,409574548292395008,"Я просто не думаю, их головы в нем. =( #INSTANTFOLLOWBACK",-1
59589,416257774264549376,"RT @katermark: @Miss_Daft Алин, этот мужчина мне спать не дает!!!((( я на всю музыку врубаю, чтобы его заглушить((((",-1
95342,422646008267685888,Снег вернулся в Россию из Америки ... А Sumerian records все ещё не предлагает мне контракт... Разочарования :-(,-1
15730,411108356510982144,"@smirnovatanyaa ахаха ну и это, а ещё мне надоело не высыпаться(",-1
88534,421499236702633984,"так и знал,что что-то с этими пельменями не так\nнечего было их есть\nживот терь сильно болит(",-1
17500,409371681967202304,"@greendayvm завалиии... он уже 3 недели ждёт, лах:D",1
88573,410823593216708608,"RT @migeruwegida: nagios, collectd или monit? с чем я проведу ночь? :)",1
94301,422409301094047744,"Да уж, сухари с хот-догом, а на вкус, как с копченой колбасой(((!!",-1
97664,422813678295977985,"по телеку такое: ""Анна Сергеевна, вы играли Снегурочку"". эээм биг бро из уочин ми? о_О",-1
31711,412966033633181696,"Не облизываю крышечки от актимеля, потому что оставляю их для Леи. А она осталась в другой стране с мамой. Скучаю очень по своей собаченьке(",-1


## очистка данных

In [9]:
data['ttext_clean'] = data['ttext'].apply(lambda t:[ w.strip() for w in t.split() if w.strip() ] )

In [10]:
data['ttext_clean'] = data['ttext_clean'].apply(
    lambda t:[ re.sub(r'^http.*',' url ', w.strip() ) for w in t  ]
  )

In [11]:
data['ttext_clean'] = data['ttext_clean'].apply(
    lambda t:[ re.sub(r'[:;]-*[)D]',' happysmile ', w.strip() )for w in t ]
  )

In [12]:
data['ttext_clean'] = data['ttext_clean'].apply(
    lambda t:[ re.sub(r'\)\)\)*',' happysmile ', w.strip() ) for w in t ]
  )

In [13]:
data['ttext_clean'] = data['ttext_clean'].apply(
    lambda t:[ re.sub(r'[:;]\*',' kisssmile ', w.strip() ) for w in t ]
  )

In [14]:
data['ttext_clean'] = data['ttext_clean'].apply(
    lambda t:[ re.sub(r':\(',' sadsmile ', w.strip() ) for w in t ]
  )

In [15]:
data['ttext_clean'] = data['ttext_clean'].apply(
    lambda t:[ re.sub(r'\(\(\(*',' sadsmile ', w.strip() ) for w in t ]
  )

In [16]:
data['ttext_clean'] = [ ' '.join(s) for s in data['ttext_clean'] ]

In [17]:
data['ttext_clean'] = data['ttext_clean'].str.lower()
data['ttext_clean'] = data['ttext_clean'].apply(lambda s: re.sub( r'\W', ' ', s))
data['ttext_clean'] = data['ttext_clean'].apply(lambda s: re.sub( r'_', ' ', s))
data['ttext_clean'] = data['ttext_clean'].apply(lambda s: re.sub( r'\b\d+\b', ' digit ', s)) 


In [18]:
data['ttext_clean'] = data['ttext_clean'].apply(lambda t:[ w.strip() for w in t.split() if w.strip() ] )

In [19]:
# замена буквенно-цифровых кодов
data['ttext_clean'] = data['ttext_clean'].apply(
    lambda t: [w for w in t if not re.match( r'\b.*\d+.*\b', w) ]
)

In [20]:
# data[['ttext_clean']]
# data[['ttext']]

---

In [21]:
with gzip.open('../data/text/stop-nltk.txt.gz','rt',encoding='utf-8') as f: 
    stopwords = set([ w.strip() for w in  f.read().split() if w.strip() ] )
ppr(stopwords)

записей: 151


In [22]:
# удаление лишних слов
data['ttext_clean'] = data['ttext_clean'].apply(lambda t:[w for w in t if w not in stopwords])

In [23]:
%xdel stopwords

In [24]:
# %%time 

# from Stemmer import Stemmer
# # pacman -S python-pystemmer
# # pip install pystemmer

# # стемминг, выделение основы слова
# data['ttext_clean'] = data['ttext_clean'].apply( lambda t:Stemmer('russian').stemWords(t) )

In [25]:
# удаление коротких слов
data['ttext_clean'] = data['ttext_clean'].apply(lambda t:[w for w in t if len(w)>2])

---

In [26]:
# data[ data['ttext_clean'].str.len()<1 ][['ttext_clean']]

In [27]:
ppr(data)
data = data[ data['ttext_clean'].str.len()>0 ].reset_index(drop=True) 
ppr(data)

записей: 226 833
записей: 226 826


In [28]:
data.sample(3)

,id,ttext,ttype,ttext_clean
115459,409054091214454784,"@Irtnka @BilanOfficial в Билановском трио, как и Волчков у Градского, набрал 120% в общей сумме. Он по голосу и мастерству рядом не стоял(",-1,"[irtnka, bilanofficial, билановском, трио, волчков, градского, набрал, digit, общей, сумме, голосу, мастерству, рядом, стоял]"
48574,410018554281328640,"Океан Эльзы – Я не сдамся без бою, девиз по жизни!)",1,"[океан, эльзы, сдамся, бою, девиз, жизни]"
77788,410713699922771968,"@KazanzevaMaria 19 баллов :)\nНо в блазен такой пздц попался.Все бы ничего, если б словарь норм был, а тема была ehrenamtliches engagement",1,"[kazanzevamaria, digit, баллов, happysmile, блазен, пздц, попался, словарь, норм, тема, ehrenamtliches, engagement]"


## строим датасет

### дополняем и перестраиваем текст

In [29]:
# vocab = ['<PAD>','<START>','<UNK>'] + sorted(set([ w for t in data_train['ttext_clean'] for w in t if w ]))
# ppr(vocab)

In [30]:
data['ttext_clean'] = data['ttext_clean'] + ['<START>']

In [31]:
n_max = data['ttext_clean'].str.len().max()
n_max

31

In [32]:
pad = ['<PAD>']*n_max

In [33]:
data[['ttext_clean']].sample(3)

,ttext_clean
124817,"[твитнуть, забуду, хотя, забуду, url, <START>]"
65198,"[allons, школы, пришла, сижу, сериал, смотрю, <START>]"
169339,"[cio, optimal, vrsoloviev, andrey, tlt, digit, разбираетесь, людях, это, святые, олигархи, радеющие, народ, свой, <START>]"


In [34]:
data['ttext_clean'] = data['ttext_clean'].apply(
    lambda t: pad[len(t):] + list(reversed(t)) 
  )

In [35]:
data[['ttext_clean']].sample(3)

,ttext_clean
155913,"[<PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <START>, sadsmile, собак, любит, фрэнк, кофе, любит, ше..."
65402,"[<PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <START>, happysmile, соснин..."
144096,"[<PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <START>, sadsmile, болииииит, сильно, очень, sadsmile, sadsmile, копч..."


In [36]:
%%time

from gensim.models.word2vec import Word2Vec

w2v_size = 256

w2v = Word2Vec( data['ttext_clean'].values, min_count=1, size=w2v_size, window=4, workers=4)

# with open('result/Word2Vec.pkl', 'wb') as f: pickle.dump(w2v, f)

CPU times: user 35.4 s, sys: 419 ms, total: 35.8 s
Wall time: 13.6 s


In [37]:
w2v_vocab = sorted([w for w in w2v.wv.vocab])
ppr(w2v_vocab)

записей: 239 404


In [38]:
data['ttext_code'] = data['ttext_clean'].apply(lambda t: [ w2v.wv.get_vector(w) for w in t ] )

### разделяем данные

In [39]:
batch_size=64

data_train = data.sample(batch_size*500).reset_index(drop=True)
ppr(data_train)

записей: 32 000


In [40]:
data_test = data[ ~data['id'].isin( data_train['id'] ) ].reset_index(drop=True)
ppr(data_test)

записей: 194 826


In [41]:
X_train = np.stack([ np.vstack(s) for s in  data_train['ttext_code'] ])
X_train.shape

(32000, 31, 256)

In [42]:
X_test = np.stack([ np.vstack(s) for s in  data_test['ttext_code'] ])
X_test.shape

(194826, 31, 256)

In [43]:
from sklearn.preprocessing import OneHotEncoder

In [44]:
y_train = data_train['ttype'].values
y_train = OneHotEncoder(categories='auto').fit_transform(y_train.reshape(-1,1) ).todense().astype(np.float32)
y_train.shape

(32000, 2)

In [45]:
y_test = data_test['ttype'].values
y_test = OneHotEncoder(categories='auto').fit_transform(y_test.reshape(-1,1) ).todense().astype(np.float32)
y_test.shape

(194826, 2)

In [46]:
# np.save('X.npy',X_train)
# np.save('y.npy',y_train)

## строим нейросеть 

In [52]:
# import numpy as np

from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Dense

In [53]:
#batch_size=64

data_dim = X_train.shape[2]
time_steps = X_train.shape[1]
num_classes = y_train.shape[1]

In [56]:
# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(512, input_shape=(time_steps, data_dim)))  
model.add(Dense(num_classes, activation='softmax'))

In [57]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [58]:
%%time

history = model.fit(X_train,y_train, batch_size=batch_size, epochs=10, )  
# validation_data=(X_val, y_val),

Epoch 1/10
32000/32000 [==============================] - 113s 4ms/step - loss: 0.4194 - acc: 0.7815
Epoch 2/10
32000/32000 [==============================] - 114s 4ms/step - loss: 0.2822 - acc: 0.8524
Epoch 3/10
32000/32000 [==============================] - 112s 4ms/step - loss: 0.2539 - acc: 0.8623
Epoch 4/10
32000/32000 [==============================] - 113s 4ms/step - loss: 0.2337 - acc: 0.8696
Epoch 5/10
32000/32000 [==============================] - 119s 4ms/step - loss: 0.2381 - acc: 0.8745
Epoch 6/10
32000/32000 [==============================] - 116s 4ms/step - loss: 0.2220 - acc: 0.8816
Epoch 7/10
32000/32000 [==============================] - 112s 4ms/step - loss: 0.2164 - acc: 0.8870
Epoch 8/10
32000/32000 [==============================] - 113s 4ms/step - loss: 0.2143 - acc: 0.8908
Epoch 9/10
32000/32000 [==============================] - 113s 4ms/step - loss: 0.2054 - acc: 0.8965
Epoch 10/10
32000/32000 [==============================] - 116s 4ms/step - loss: 0.2011 - a

---

In [59]:
%%time

results = model.evaluate(X_test,y_test)

print(results)

194826/194826 [==============================] - 375s 2ms/step
[0.2127477115502639, 0.8935460359506391]


---

In [60]:
# history_dict = history.history
# history_dict.keys()

In [61]:
# # import matplotlib.pyplot as plt

# acc = history.history['acc']
# val_acc = history.history['val_acc']
# loss = history.history['loss']
# val_loss = history.history['val_loss']

# epochs = range(1, len(acc) + 1)

# # "bo" is for "blue dot"
# plt.plot(epochs, loss, 'bo', label='Training loss')
# # b is for "solid blue line"
# plt.plot(epochs, val_loss, 'b', label='Validation loss')
# plt.title('Training and validation loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()

# plt.show()

In [62]:
# plt.clf()   # clear figure
# acc_values = history_dict['acc']
# val_acc_values = history_dict['val_acc']

# plt.plot(epochs, acc, 'bo', label='Training acc')
# plt.plot(epochs, val_acc, 'b', label='Validation acc')
# plt.title('Training and validation accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.legend()

# plt.show()

---

In [ ]:
# from keras.preprocessing import sequence
# from keras.utils import np_utils
# from keras.models import Sequential
# from keras.layers.core import Dense, Dropout, Activation
# from keras.layers.embeddings import Embedding
# from keras.layers.recurrent import LSTM

In [ ]:
# max_features = 100000
# maxlen = X.shape[0]
# # batch_size = 32

# model = Sequential()
# model.add(Embedding(max_features, 128, input_length=maxlen))
# # model.add(LSTM(64, return_sequences=True))
# model.add(LSTM(64))
# # model.add(Dropout(0.5))
# model.add(Dense(2))
# model.add(Activation('sigmoid'))

In [ ]:
# model.compile(loss='binary_crossentropy',
#               optimizer='adam',
#               class_mode="binary")

In [ ]:
# model.fit(
#     X, y, 
#     batch_size=batch_size, 
#     nb_epoch=1 # , show_accuracy=True
# )

In [ ]:
# result = model.predict_proba(X)

---

In [ ]:
# import numpy as np

# from keras.models import Sequential
# from keras.layers import LSTM
# from keras.layers import Dense

In [ ]:
# data_dim = 16
# timesteps = 8
# num_classes = 2

# num_ex = 1000

# x_train = np.random.random((num_ex, timesteps, data_dim))
# y_train = np.random.randint(1,3,num_ex)

# x_train.shape

# # [ пример, элемент посл., вектор ]

In [ ]:
# from sklearn.preprocessing import OneHotEncoder

# y_train = np.random.randint(1,3,num_ex)
# y_train = OneHotEncoder(categories='auto').fit_transform(y_train.reshape(-1,1) ).todense()
# y_train.shape

In [ ]:
# # expected input data shape: (batch_size, timesteps, data_dim)
# model = Sequential()

# # returns a sequence of vectors of dimension 32
# model.add(LSTM(32,return_sequences=True,input_shape=(timesteps, data_dim)))  

# # returns a sequence of vectors of dimension 32
# model.add(LSTM(32,return_sequences=True))  

# model.add(LSTM(32))  # return a single vector of dimension 32

# model.add(Dense(num_classes, activation='softmax'))

In [ ]:
# # expected input data shape: (batch_size, timesteps, data_dim)
# model = Sequential()

# # returns a sequence of vectors of dimension 32
# model.add(LSTM(32,input_shape=(timesteps, data_dim)))  

# model.add(Dense(num_classes, activation='softmax'))

In [ ]:
# model.compile(loss='categorical_crossentropy',
#               optimizer='rmsprop',
#               metrics=['accuracy'])

In [ ]:
# model.fit(x_train, y_train,
#           batch_size=64, epochs=115,
#           # validation_data=(x_val, y_val)
#          )